##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 혼합 정밀도

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/mixed_precision"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/mixed_precision.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/mixed_precision.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/mixed_precision.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a>
  </td>
</table>

## 개요

Mixed precision is the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory. By keeping certain parts of the model in the 32-bit types for numeric stability, the model will have a lower step time and train equally as well in terms of the evaluation metrics such as accuracy. This guide describes how to use the Keras mixed precision API to speed up your models. Using this API can improve performance by more than 3 times on modern GPUs and 60% on TPUs.

오늘날 대부분의 모델은 32-bit 메모리를 사용하는 float32 dtype을 사용합니다. 그러나 정밀도가 낮은 dtype인 float16과 bfloat16도 있으며 각각은 16-bit의 메모리를 사용합니다. 최신 가속기는 16-bit 계산을 실행할 수 있는 특수한 하드웨어가 있어 16-bit dtype을 메모리에서 더 빨리 읽을 수 있으므로 16-bit dtype에서 연산을 더 빠르게 실행할 수 있습니다.

NVIDIA GPU는 float32보다 float16에서 연산을 더 빠르게 실행할 수 있고 TPU는 float32보다 bfloat16에서 연산을 더 빨리 실행할 수 있습니다. 따라서 이러한 정밀도가 낮은 dtype은 해당 장치에서 가능할 때마다 사용해야 합니다. 그러나 변수와 몇 가지 계산은 여전히 숫자로 인해 float32에 있어야 모델이 동일한 품질로 훈련합니다. Keras 혼합 정밀도 API를 사용하면 float16 또는 bfloat16을 float32와 함께 사용하여 float16/bfloat16의 성능 이점과 float32의 수치 안정성 이점을 얻을 수 있습니다.

Note: In this guide, the term "numeric stability" refers to how a model's quality is affected by the use of a lower-precision dtype instead of a higher precision dtype. An operation is "numerically unstable" in float16 or bfloat16 if running it in one of those dtypes causes the model to have worse evaluation accuracy or other metrics compared to running the operation in float32.

## 설정

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision

## 지원하는 하드웨어

혼합 정밀도는 대부분의 하드웨어에서 실행되지만 최신 NVIDIA GPU 및 Cloud TPU에서는 모델의 속도만 향상됩니다. NVIDIA GPU는 float16과 float32의 혼합 사용을 지원하는 반면 TPU는 bfloat16과 float32의 혼합을 지원합니다.

Among NVIDIA GPUs, those with compute capability 7.0 or higher will see the greatest performance benefit from mixed precision because they have special hardware units, called Tensor Cores, to accelerate float16 matrix multiplications and convolutions. Older GPUs offer no math performance benefit for using mixed precision, however memory and bandwidth savings can enable some speedups. You can look up the compute capability for your GPU at NVIDIA's [CUDA GPU web page](https://developer.nvidia.com/cuda-gpus). Examples of GPUs that will benefit most from mixed precision include RTX GPUs, the V100, and the A100.

참고: Google Colab에서 이 가이드를 실행하면 일반적으로 GPU 런타임에 P100이 연결됩니다. P100은 컴퓨팅 능력 6.0을 갖추고 있으며 눈에띄는 속도 향상을 기대하지는 않습니다.

다음을 통해 GPU 타입을 확인할 수 있습니다. 이 명령은 NVIDIA 드라이버가 설치된 경우에만 존재하므로 다음과 같은 경우 오류가 발생합니다.

In [ ]:
!nvidia-smi -L

모든 Cloud TPU는 bfloat16을 지원합니다.

Even on CPUs and older GPUs, where no speedup is expected, mixed precision APIs can still be used for unit testing, debugging, or just to try out the API. On CPUs, mixed precision will run significantly slower, however.

## dtype 정책 설정

To use mixed precision in Keras, you need to create a `tf.keras.mixed_precision.Policy`, typically referred to as a *dtype policy*. Dtype policies specify the dtypes layers will run in. In this guide, you will construct a policy from the string `'mixed_float16'` and set it as the global policy. This will cause subsequently created layers to use mixed precision with a mix of float16 and float32.

In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

For short, you can directly pass a string to `set_global_policy`, which is typically done in practice.

In [ ]:
# Equivalent to the two lines above
mixed_precision.set_global_policy('mixed_float16')

이 정책은 레이어의 두 가지 중요한 측면, 즉 레이어 계산이 수행되는 dtype과 레이어 변수의 dtype을 지정합니다. 위에서 `mixed_float16` 정책을 만들었습니다(예, 문자열 `'mixed_float16'`을 생성자에 전달하여 만든 `mixed_precision.Policy`). 이 정책에서 레이어는 float16 계산 및 float32 변수를 사용합니다. 계산은 성능을 위해 float16에서 수행되지만, 수치 안정성을 위해서는 변수를 float32로 유지해야 합니다. 이러한 정책 속성을 직접 쿼리 할 수 있습니다.

In [ ]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

앞에서 언급했듯이 `mixed_float16` 정책은 7.0 이상의 컴퓨팅 능력을 갖춘 NVIDIA GPU의 성능이 가장 크게 향상됩니다. 이 정책은 다른 GPU 및 CPU에서 실행되지만, 성능이 향상되지 않을 수 있습니다. TPU의 경우 `mixed_bfloat16` 정책을 대신 사용해야 합니다.

## 모델 구축

다음으로 간단한 모델을 만들어 봅시다. TensorFlow 런타임의 오버헤드가 일반적으로 실행 시간을 지배하여 GPU의 성능 향상을 무시할 수 있기 때문에 매우 작은 토이 모델은 일반적으로 혼합 정밀도의 이점을 얻지 못합니다. 따라서 GPU를 사용하는 경우 각각 4096개의 유닛으로 두 개의 큰 `Dense` 레이어를 만들어 봅시다.

In [ ]:
inputs = keras.Input(shape=(784,), name='digits')
if tf.config.list_physical_devices('GPU'):
  print('The model will run with 4096 units on a GPU')
  num_units = 4096
else:
  # Use fewer units on CPUs so the model finishes in a reasonable amount of time
  print('The model will run with 64 units on a CPU')
  num_units = 64
dense1 = layers.Dense(num_units, activation='relu', name='dense_1')
x = dense1(inputs)
dense2 = layers.Dense(num_units, activation='relu', name='dense_2')
x = dense2(x)

각 계층에는 정책이 있으며 기본적으로 전역 정책을 사용합니다. 따라서 이전에 전역 정책을 `mixed_float16`으로 설정했기 때문에 각 `Dense` 레이어는 `mixed_float16` 정책을 갖습니다. 이로 인해 밀도가 높은 레이어는 float16 계산을 수행하고 float32 변수를 갖게됩니다. float16 계산을 수행하기 위해 입력을 float16으로 캐스팅하여 결과적으로 출력이 float16이됩니다. 변수는 float32이며 dtype 불일치로 인한 오류를 피하기 위해 레이어를 호출하면 float16으로 캐스팅됩니다.

In [ ]:
print(dense1.dtype_policy)
print('x.dtype: %s' % x.dtype.name)
# 'kernel' is dense1's variable
print('dense1.kernel.dtype: %s' % dense1.kernel.dtype.name)

다음으로 출력 예측을 작성합니다. 일반적으로 다음과 같이 출력 예측을 작성할 수 있지만 float16에서는 항상 수치상으로 안정적이지는 않습니다.

In [ ]:
# INCORRECT: softmax and model output will be float16, when it should be float32
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

모델의 끝에서 softmax 활성화는 float32이어야 합니다. dtype 정책이 `mixed_float16`이므로, softmax 활성화는 일반적으로 float16 계산 dtype을 가지며 float16 텐서를 출력합니다.

Dense와 softmax 레이어를 분리하고 `dtype='float32'`를 softmax 레이어에 전달하는 것으로 이 문제를 해결할 수 있습니다.

In [ ]:
# CORRECT: softmax and model output are float32
x = layers.Dense(10, name='dense_logits')(x)
outputs = layers.Activation('softmax', dtype='float32', name='predictions')(x)
print('Outputs dtype: %s' % outputs.dtype.name)

`dtype='float32'`를 softmax 레이어 생성자에 전달하면 레이어의 dtype 정책이 `float32` 정책으로 재정의되어 계산을 수행하고 변수를 float32로 유지합니다. 마찬가지로 `dtype=mixed_precision.Policy('float32')`대신 전달할 수도 있습니다. 레이어는 항상 dtype 인수를 정책으로 변환합니다. `Activation` 레이어에는 변수가 없으므로 정책의 변수 dtype은 무시되지만 float32 정책의 계산 dtype은 softmax 및 모델 출력을 float32로 만듭니다.

모델의 중간에 float16 softmax를 추가하는 것은 괜찮지만 모델 끝의 softmax는 float32로 있어야합니다. 그 이유는 softmax에서 손실까지 흐르는 중간 텐서가 float16 또는 bfloat16이면 숫자 문제가 발생할 수 있기 때문입니다.

float16 계산이 수치상으로 안정적이지 않다고 생각된다면 `dtype='float32'`을 전달하여 모든 레이어의 dtype을 float32로 재정의할 수 있습니다. 하지만 일반적으로 대부분의 레이어는 `mixed_float16` 및 `mixed_bfloat16`으로 충분한 정밀도를 갖기 때문에 모델의 마지막 레이어에서만 필요합니다.

모델이 softmax로 끝나지 않더라도 출력은 여전히 float32이어야 합니다. 이 특정 모델에는 필요하지 않지만, 다음을 사용하여 모델 출력을 float32로 캐스팅 할 수 있습니다.

In [ ]:
# The linear activation is an identity function. So this simply casts 'outputs'
# to float32. In this particular case, 'outputs' is already float32 so this is a
# no-op.
outputs = layers.Activation('linear', dtype='float32')(outputs)

Next, finish and compile the model, and generate input data:

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

이 예제는 입력 데이터를 int8에서 float32로 캐스팅합니다. 255로 나누기가 CPU에 있으며 float16 연산은 float32 연산보다 느리기 때문에 float16으로 캐스팅하지 않습니다. 이 경우 성능 차이는 무시할만하지만, 일반적으로 CPU에서 실행되는 경우 float32에서 입력 처리 계산을 실행해야 합니다. 각 레이어는 부동 소수점 입력을 계산 dtype에 캐스팅하므로 모델의 첫 번째 레이어는 입력을 float16으로 캐스팅합니다.

모델의 초기 가중치가 검색됩니다. 가중치를 로딩하여 처음부터 다시 훈련 할 수 있습니다.

In [ ]:
initial_weights = model.get_weights()

## Model.fit으로 모델 훈련

Next, train the model:

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=8192,
                    epochs=5,
                    validation_split=0.2)
test_scores = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])


Notice the model prints the time per step in the logs: for example, "25ms/step". The first epoch may be slower as TensorFlow spends some time optimizing the model, but afterwards the time per step should stabilize.

If you are running this guide in Colab, you can compare the performance of mixed precision with float32. To do so, change the policy from `mixed_float16` to `float32` in the "Setting the dtype policy" section, then rerun all the cells up to this point. On GPUs with compute capability 7.X, you should see the time per step significantly increase, indicating mixed precision sped up the model.  Make sure to change the policy back to `mixed_float16` and rerun the cells before continuing with the guide.

많은 실제 모델의 경우, float16 텐서가 메모리의 절반을 차지하므로 혼합 정밀도를 사용하면 메모리 부족없이 배치 크기를 두 배로 늘릴 수 있습니다. 그러나 각 배치가 60,000개의 이미지로 구성된 전체 MNIST 데이터세트로 구성된 모든 dtype에서 모델을 실행할 수 있으므로, 이 토이 모델에는 적용되지 않습니다.

If running mixed precision on a TPU, you will not see as much of a performance gain compared to running mixed precision on GPUs, especially pre-Ampere GPUs. This is because TPUs do certain ops in bfloat16 under the hood even with the default dtype policy of float32. This is similar to how Ampere GPUs use TensorFloat-32 by default. Compared to Ampere GPUs, TPUs typically see less performance gains with mixed precision on real-world models.

많은 실제 모델의 경우, float16 텐서가 메모리의 절반을 차지하므로 혼합 정밀도를 사용하면 메모리 부족없이 배치 크기를 두 배로 늘릴 수 있습니다. 그러나 각 배치가 60,000개의 이미지로 구성된 전체 MNIST 데이터세트로 구성된 모든 dtype에서 모델을 실행할 수 있으므로, 이 토이 모델에는 적용되지 않습니다.

## 손실 조정

Loss scaling is a technique which `tf.keras.Model.fit` automatically performs with the `mixed_float16` policy to avoid numeric underflow. This section describes what loss scaling is and the next section describes how to use it with a custom training loop.

### 언더플로우 및 오버플로우

float16 데이터 형식은 float32에 비해 동적 범위가 좁습니다. 이는 $65504$ 이상의 값은 무한대로 오버플로우되고 $6.0 \times 10^{-8}$ 미만의 값은 0으로 언더플로우됨을 의미합니다. float32 및 bfloat16은 동적 범위가 훨씬 높아 오버플로우 및 언더플로우가 문제가 되지 않습니다.

예:

In [ ]:
x = tf.constant(256, dtype='float16')
(x ** 2).numpy()  # Overflow

In [ ]:
x = tf.constant(1e-5, dtype='float16')
(x ** 2).numpy()  # Underflow

실제로 float16으로 오버플로우가 거의 발생하지 않습니다. 또한 순방향 전달에서 언더플로우가 거의 발생하지 않습니다. 그러나 역방향 전달동안 그래디언트가 0으로 언더플로우 될 수 있습니다. 손실 조정은 이러한 언더플로우를 방지하는 기술입니다.

### Loss scaling overview

The basic concept of loss scaling is simple: simply multiply the loss by some large number, say $1024$, and you get the *loss scale* value. This will cause the gradients to scale by $1024$ as well, greatly reducing the chance of underflow. Once the final gradients are computed, divide them by $1024$ to bring them back to their correct values.

이 프로세스의 의사 코드는 다음과 같습니다.

```
loss_scale = 1024
loss = model(inputs)
loss *= loss_scale
# Assume `grads` are float32. You do not want to divide float16 gradients.
grads = compute_gradient(loss, model.trainable_variables)
grads /= loss_scale
```

손실 규모를 선택하는 것은 까다로울 수 있습니다. 손실 규모가 너무 낮으면 그래디언트가 여전히 0으로 언더플로우일 수 있습니다. 너무 높으면 반대의 문제가 발생합니다. 그래디언트가 무한대로 오버플로우일 수 있습니다.

To solve this, TensorFlow dynamically determines the loss scale so you do not have to choose one manually. If you use `tf.keras.Model.fit`, loss scaling is done for you so you do not have to do any extra work. If you use a custom training loop, you must explicitly use the special optimizer wrapper `tf.keras.mixed_precision.LossScaleOptimizer` in order to use loss scaling. This is described in the next section.


## 사용자 지정 훈련 루프를 사용하여 모델 훈련

So far, you have trained a Keras model with mixed precision using `tf.keras.Model.fit`. Next, you will use mixed precision with a custom training loop. If you do not already know what a custom training loop is, please read the [Custom training guide](../tutorials/customization/custom_training_walkthrough.ipynb) first.

혼합 정밀도로 사용자 지정 훈련 루프를 실행하려면 float32에서 실행하는것으로부터 두 가지 변경이 필요합니다.

1. 혼합 정밀도로 모델을 작성합니다 (이미 수행했습니다)
2. `mixed_float16` 을 사용하는 경우 손실 조정을 명시적으로 사용합니다.


For step (2), you will use the `tf.keras.mixed_precision.LossScaleOptimizer` class, which wraps an optimizer and applies loss scaling. By default, it dynamically determines the loss scale so you do not have to choose one. Construct a `LossScaleOptimizer` as follows.

In [ ]:
optimizer = keras.optimizers.RMSprop()
optimizer = mixed_precision.LossScaleOptimizer(optimizer)

If you want, it is possible choose an explicit loss scale or otherwise customize the loss scaling behavior, but it is highly recommended to keep the default loss scaling behavior, as it has been found to work well on all known models. See the `tf.keras.mixed_precision.LossScaleOptimizer` documention if you want to customize the loss scaling behavior.

Next, define the loss object and the `tf.data.Dataset`s:

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
train_dataset = (tf.data.Dataset.from_tensor_slices((x_train, y_train))
                 .shuffle(10000).batch(8192))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(8192)

Next, define the training step function. You will use two new methods from the loss scale optimizer to scale the loss and unscale the gradients:

- `get_scaled_loss(loss)`: 손실에 손실 규모를 곱합니다
- `get_unscaled_gradients(gradients)`: 조정된 그래디언트 목록을 입력으로 취하고 각각을 손실 규모로 나누어 조정을 해제합니다

그래디언트의 언더플로우를 방지하려면 이러한 기능을 사용해야 합니다. `LossScaleOptimizer.apply_gradients`는 Infs 또는 NaN이 없는 그래디언트를 적용합니다. 이는 또한 손실 규모를 업데이트하는데, 그래디언트에 Infs 또는 NaN이 있으면 손실 규모를 절반으로 줄이고 그렇지 않은 경우 잠재적으로 증가시킵니다.

In [ ]:
@tf.function
def train_step(x, y):
  with tf.GradientTape() as tape:
    predictions = model(x)
    loss = loss_object(y, predictions)
    scaled_loss = optimizer.get_scaled_loss(loss)
  scaled_gradients = tape.gradient(scaled_loss, model.trainable_variables)
  gradients = optimizer.get_unscaled_gradients(scaled_gradients)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

`LossScaleOptimizer`는 훈련 시작 시 처음 몇 단계를 건너뛸 수 있습니다. 최적의 손실 규모를 신속하게 결정할 수 있도록 손실 규모가 크게 시작됩니다. 몇 단계를 거치면 손실 규모가 안정화되고 몇 단계만 건너뜁니다. 이 프로세스는 자동으로 수행되며 훈련 품질에는 영향을 미치지 않습니다.

Now, define the test step:


In [ ]:
@tf.function
def test_step(x):
  return model(x, training=False)

Load the initial weights of the model, so you can retrain from scratch:

In [ ]:
model.set_weights(initial_weights)

Finally, run the custom training loop:

In [ ]:
for epoch in range(5):
  epoch_loss_avg = tf.keras.metrics.Mean()
  test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      name='test_accuracy')
  for x, y in train_dataset:
    loss = train_step(x, y)
    epoch_loss_avg(loss)
  for x, y in test_dataset:
    predictions = test_step(x)
    test_accuracy.update_state(y, predictions)
  print('Epoch {}: loss={}, test accuracy={}'.format(epoch, epoch_loss_avg.result(), test_accuracy.result()))

## GPU 성능 팁

다음은 GPU에서 혼합 정밀도를 사용할 때의 성능 팁입니다.

### 배치 크기 늘리기

모델 품질에 영향을 미치지 않으면 혼합 정밀도를 사용할 때 배치 크기를 두 배로 실행합니다. float16 텐서가 절반의 메모리를 사용하므로 메모리 부족없이 배치 크기를 두 배로 늘릴 수 있습니다. 배치 크기를 늘리면 일반적으로 훈련 처리량, 즉 모델을 실행할 수 있는 초당 훈련 요소가 증가합니다.

### GPU Tensor Cores 사용 보장

앞에서 언급했듯이 최신 NVIDIA GPU는 float16 행렬을 매우 빠르게 곱할 수 있는 Tensor Cores라는 특수 하드웨어 장치를 사용합니다. 그러나 Tensor Cores는 텐서의 특정 크기가 8의 배수 이어야 합니다. 다음 예에서는 텐서 코어를 사용하기 위해 8의 배수 이어야하는 경우에만 인수가 굵게 표시됩니다.

- tf.keras.layers.Dense(**units=64**)
- tf.keras.layers.Conv2d(**filters=48**, kernel_size=7, stride=3)
    - tf.keras.layers.Conv3d와 같은 다른 컨볼루션 레이어와 유사합니다
- tf.keras.layers.LSTM(**units=64**)
    - tf.keras.layers.GRU와 같은 다른 RNN과 유사합니다
- tf.keras.Model.fit(epochs=2, **batch_size=128**)

가능하면 Tensor Cores를 사용하십시오. 자세한 내용을 보려면 [NVIDIA 딥 러닝 성능 안내서](https://docs.nvidia.com/deeplearning/sdk/dl-performance-guide/index.html)에 Tensor Cores 및 기타 Tensor Cores 관련 성능 정보를 사용하기위한 정확한 요구 사항이 설명되어 있습니다.

### XLA

XLA is a compiler that can further increase mixed precision performance, as well as float32 performance to a lesser extent. Refer to the [XLA guide](https://www.tensorflow.org/xla) for details.

## Cloud TPU 성능 팁

As with GPUs, you should try doubling your batch size when using Cloud TPUs because bfloat16 tensors use half the memory. Doubling batch size may increase training throughput.

TPUs do not require any other mixed precision-specific tuning to get optimal performance. They already require the use of XLA. TPUs benefit from having certain dimensions being multiples of $128$, but this applies equally to the float32 type as it does for mixed precision. Check the [Cloud TPU performance guide](https://cloud.google.com/tpu/docs/performance-guide) for general TPU performance tips, which apply to mixed precision as well as float32 tensors.

## 요약

- 컴퓨팅 능력이 7.0 이상인 TPU 또는 NVIDIA GPU를 사용하는 경우 성능이 최대 3배 향상되므로 혼합 정밀도를 사용해야 합니다.

- 다음과 같이 혼합 정밀도를 사용할 수 있습니다.

    ```python
    # On TPUs, use 'mixed_bfloat16' instead
    mixed_precision.set_global_policy('mixed_float16')
    ```

- 모델이 softmax로 끝나는 경우 float32인지 확인합니다. 모델이 무엇으로 끝나는지에 관계없이 출력이 float32인지 확인합니다.
- If you use a custom training loop with `mixed_float16`, in addition to the above lines, you need to wrap your optimizer with a `tf.keras.mixed_precision.LossScaleOptimizer`. Then call `optimizer.get_scaled_loss` to scale the loss, and `optimizer.get_unscaled_gradients` to unscale the gradients.
- 평가 정확도가 떨어지지 않으면 훈련 배치 크기를 두 배로 늘립니다
- GPU에서 성능을 최대화하려면 대부분의 텐서 크기가 $8$의 배수가 되도록 합니다

For an example of mixed precision using the `tf.keras.mixed_precision` API, check [functions and classes related to training performance](https://github.com/tensorflow/models/blob/master/official/modeling/performance.py). Check out the official models, such as [Transformer](https://github.com/tensorflow/models/blob/master/official/nlp/modeling/layers/transformer_encoder_block.py), for details.
